In [ ]:
# Для вас представлена задача, сделанная на открытых данных.

# Вам нужно предсказать цену товара по другим характеристикам.

# Метрика качества RMSLE 

# np.sqrt(np.mean((np.log(y_test + 1) - np.log(y_hat + 1))**2))
# np.sqrt(np.mean(np.power(np.log1p(y_test)-np.log1p(y_hat), 2)))
# np.sqrt(mean_squared_log_error(y_test, predictions))

# can_buy - возможно ли купить этот товар,

# can_promote - возможность продвигать товар,

# category - категория товара,

# contacts_visible - видны ли контакты,

# date_created - таймстемп создания товара,

# delivery_available - возможность доставки,

# description - описание,

# fields - свойства товара,

# id - идентификатор продукта,

# images - id картинок,

# location - местрорасположение,

# mortgage_available - возможность ипотеки,

# name - навзаник,

# payment_available - возможность онлайн оплаты,

# price - цена,

# subcategory - подкатегория,

# subway - метро,

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pickle
import seaborn as sns
import datetime as dt
warnings.filterwarnings('ignore')

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [61]:
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import validation_curve
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/alekum/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
STOPWORDS = []
STOPWORDS.extend(stopwords.words('english'))
STOPWORDS.extend(stopwords.words('russian'))

In [81]:
import pymorphy2
morph  = pymorphy2.MorphAnalyzer()
# will care only about stopwords, neither isalnum or is_digit
morphy_tokenizer = lambda text: [morph.parse(w)[0].normal_form for w in text.split() if w not in STOPWORDS] 

In [82]:
def nltk_tokenizer(text):
    return [w for w in nltk.word_tokenize(text) if w.isalnum() and w not in STOPWORDS]

In [5]:
# np.sqrt(np.mean((np.log(y + 1) - np.log(y_hat + 1))**2))    
# np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y_hat), 2))) 
# np.sqrt(mean_squared_log_error(y, y_hat)) 
def RMLSE_SCORE(y, y_hat):
    return np.sqrt(mean_squared_log_error(y, y_hat)) 

# E - model. estimator 
# X - data
# y - target
def RMLSE_SCORER(E, X, y):
    return RMLSE_SCORE(y, E.predict(X))

In [6]:
# Get rid of from all non helpful things from the text: по, (текст какой-то в скобках), slashes and etc
import re
re_parentheses_trash_ripper = re.compile("[!#$%&'*+.,^_`|~:;]|\(.*?\)", flags=re.IGNORECASE)
re_slash_dash_replacer = re.compile("[-/[\\]]")
re_double_quotes_ripper = re.compile("\".*?\"")
re_html_escapers = re.compile("&\w+;")
re_punctuation_ripper = re.compile("[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]", flags=re.IGNORECASE)

def erase_chars_by_pattern(_text, _re_compiled_pattern, _substitution=''):
    _text = re.sub(_re_compiled_pattern, 
                   _substitution, 
                   str(_text)).strip()
    return _text

def name_descr_preproc(df):
    # I am too far away from doing it more efficiently putting it into pipeline(loop) with line of the parameters
    # so, suck it up=)
    df['name'] = df['name'].map(lambda value: erase_chars_by_pattern(value, re_parentheses_trash_ripper))
    df['name'] = df['name'].map(lambda value: erase_chars_by_pattern(value, re_slash_dash_replacer, ' '))
    df['name'] = df['name'].map(lambda value: erase_chars_by_pattern(value, re_double_quotes_ripper))
    df['name'] = df['name'].map(lambda value: str(value).lower())
    
    df['description'] = df['description'].map(lambda value: erase_chars_by_pattern(value, re_slash_dash_replacer, ' '))
    df['description'] = df['description'].map(lambda value: erase_chars_by_pattern(value, re_punctuation_ripper))
    df['description'] = df['description'].map(lambda value: erase_chars_by_pattern(value, re_html_escapers))
    df['description'] = df['description'].map(lambda value: str(value).lower())
    df['description'] = df['description'].map(lambda value: ' '.join([word for word in value.split() if word.isalnum()]))
    
    return df

In [7]:
def load_pd_from_pickle(pickle_file=None):
    assert pickle_file
    return pickle.load(open(pickle_file, 'rb'))

In [8]:
train_sample_pd = load_pd_from_pickle('train_sample.pckl')

In [9]:
train_sample_pd.shape

(100000, 17)

In [10]:
# f, ax = plt.subplots(figsize = (21, 11))
# sns.heatmap(train_sample_pd.corr(), ax = ax, annot=True)

In [11]:
train_sample_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 42754
Data columns (total 17 columns):
can_buy               100000 non-null bool
can_promote           100000 non-null bool
category              100000 non-null int64
contacts_visible      100000 non-null bool
date_created          100000 non-null int64
delivery_available    100000 non-null bool
description           100000 non-null object
fields                100000 non-null object
id                    100000 non-null object
images                100000 non-null object
location              100000 non-null object
mortgage_available    100000 non-null bool
name                  100000 non-null object
payment_available     100000 non-null bool
price                 100000 non-null int64
subcategory           100000 non-null int64
subway                16 non-null object
dtypes: bool(6), int64(4), object(7)
memory usage: 9.7+ MB


In [12]:
train_sample_pd.sample(n=4).T

,19274,85391,55460,25060
can_buy,False,False,False,False
can_promote,False,False,False,False
category,14,1,22,13
contacts_visible,True,True,True,True
date_created,1488645519,1493647832,1480136069,1485691245
delivery_available,False,False,False,False
description,Набором дешевле!в него входит тени 88 оттенков...,Продам ВАЗ 2104 в хорошем техническом состояни...,"На синтепоне, подклад хб","НОВЫЙ Samsung UE55KS8005, В ЗАПЕЧАТАННОЙ УПАКО..."
fields,"[{'field': {'name': 'Красота и здоровье', 'id'...","[{'field': {'name': 'Транспорт и автотовары', ...","[{'field': {'name': 'Детский гардероб', 'id': ...","[{'field': {'name': 'Электроника', 'id': 13, '..."
id,348585328d40979cf8deab85,3c4a0381562ddb3c8d147095,27e2fde5b648da6958519385,cb09ca183ce20368d69dd885
images,"[{'id': '58baecbee57ad4e467377578', 'num': 1, ...","[{'id': '5a800421ec985562ea44e683', 'num': 1, ...","[{'id': '58391582080cbd5678f6347e', 'num': 1, ...","[{'id': '588dd96dd9f65ada1bacb816', 'num': 1, ..."


In [13]:
train_sample_pd[train_sample_pd['subway'].notnull()].loc[:,['subway']].values[:1]

array([[{'name': 'Октябрьская', 'color': '', 'icon': {'url': {'ios': 'https://api.youla.iohttps://api.youla.io/files/icons/subway/M_ED1B35.png', 'and': 'https://api.youla.iohttps://api.youla.io/files/icons/subway/M_ED1B35.png', 'web_svg': 'https://api.youla.iohttps://api.youla.io/files/icons/subway/M_ED1B35.svg'}}, 'distance': ''}]],
      dtype=object)

In [14]:
train_sample_pd[train_sample_pd['fields'].notnull()].loc[:,['fields']].values[:1]

array([[list([{'field': {'name': 'Женский гардероб', 'id': 9, 'slug': 'zhenskaya-odezhda', 'slug_id': 'zhenskaya-odezhda', 'icon': {'url': {'and': 'https://api.youla.io/files/icons/icon-cat-wclothes5x.png', 'ios': 'https://api.youla.io/files/icons/icon-cat-wclothes-dark4x.png', 'ios_color': 'https://api.youla.io/files/icons/icon-cat-wclothes4x.png', 'web_svg': 'https://api.youla.io/files/icons/icon-cat-wclothes.svg'}}, 'type': 'category'}, 'order': 1, 'values': None}, {'field': {'name': 'Аксессуары', 'slug': 'aksessuary', 'slug_id': 'zhenskaya-odezhda-aksessuary', 'id': 914, 'parent_id': 9, 'icon': None, 'editable': True, 'default': False, 'type': 'subcategory'}, 'order': 2, 'values': None}, {'field': {'id': 4028, 'name': 'Тип', 'description': '', 'field_type': 'multiselect', 'slug_id': 'zhenskaya_odezhda_aksessuary_tip', 'type': 'additional_field'}, 'order': 13, 'values': [{'selected_value': {'id': 4042, 'value': 'Сумки'}}]}, {'field': {'id': 3021, 'name': 'Цвет', 'description': 'Выбе

In [15]:
train_sample_pd[train_sample_pd['fields'].notnull()].loc[:10,['category', 'subcategory', 'name','fields']]

,category,subcategory,name,fields
0,9,914,Сумка DG,"[{'field': {'name': 'Женский гардероб', 'id': ..."
1,22,2202,Комплект,"[{'field': {'name': 'Детский гардероб', 'id': ..."
2,22,2204,Пальтишко демисезонное,"[{'field': {'name': 'Детский гардероб', 'id': ..."
3,22,2209,Attipas,"[{'field': {'name': 'Детский гардероб', 'id': ..."
5,22,2204,Жилет теплый,"[{'field': {'name': 'Детский гардероб', 'id': ..."
6,9,902,"Босоножки, сабо","[{'field': {'name': 'Женский гардероб', 'id': ..."
7,15,1508,Жесткий диск для ноутбука Seagate 320gb,"[{'field': {'name': 'Компьютерная техника', 'i..."
8,4,403,Ковер 324 на 240,"[{'field': {'name': 'Для дома и дачи', 'id': 4..."
10,3,301,Кенгуру,"[{'field': {'name': 'Детские товары', 'id': 3,..."


In [16]:
train_sample_pd['description'] = train_sample_pd['description'].astype(str).fillna('ПУСТО')

In [17]:
for i, c in enumerate(train_sample_pd.select_dtypes(include='bool').columns):
    print("=="*10 + c)
    print(train_sample_pd[c].value_counts())
    print("n\n")

====================can_buy
False    100000
Name: can_buy, dtype: int64
n

====================can_promote
False    100000
Name: can_promote, dtype: int64
n

====================contacts_visible
True    100000
Name: contacts_visible, dtype: int64
n

====================delivery_available
False    99707
True       293
Name: delivery_available, dtype: int64
n

====================mortgage_available
False    100000
Name: mortgage_available, dtype: int64
n

====================payment_available
True     54892
False    45108
Name: payment_available, dtype: int64
n



In [18]:
def prepare_data(df):
    # DROP LIST:
    # `subway`                      it is not fully filled - ?!!!!
    # `location`                    it is not meaningful
    # `mortgage_available`          it is not meaningful
    # `can_buy`, `can_promote`, `contacts_visible`
    df = df.drop(['mortgage_available' , 
                 'can_buy', 
                 'can_promote', 
                 'contacts_visible',
                 'subway', 
                 'location'], axis=1)
    
    # images, perhaps it is not relevant, but ???
    # df['is_images'] = np.where(df['images'].notnull(), 1, 0)
    df = df.drop(['images'], axis=1)
    
    # get the price in rubles
    df['price'] = df['price'] / 100 
    
    df['description'] = df['description'].map(lambda v: 'Не указано' if not str(v) else str(v))
    
    # funny stupid(probably) action, substruct 21 period
    df['date_created'] = np.log(df['date_created']) - 21 # take my risk here, I am not sured that 21 is correct! for whole dataset
    
    # convert fields
    def calculate(v):
        return v[0]['field']['name'], v[1]['field']['name']

    # @NOTE: Combine texts into one big BAG of WORDS!
    df['fields_g1'],df['fields_g2'] = zip(*df["fields"].map(calculate))
    df = df.drop(['fields'], axis=1)
    
    
    # convert False = 0, True = 1 
    mapper = {False:0, True:1}
    for col in df.select_dtypes(include='bool').columns:
        df[col] = df[col].replace(mapper)
    
    
    
    return df

In [19]:
c_df = train_sample_pd.pipe(prepare_data)

In [20]:
c_df.sample(n=4).T

,22416,61445,50182,30077
category,3,22,3,1
date_created,0.119737,0.111257,0.11283,0.112257
delivery_available,0,0,0,0
description,Игрушка новая состояние отличное если что скин...,"Состояние отличное, одевали раза 3, мягкий, ми...","Костюм для занятий танцами, гимнастикой, хорео...",BRIDGESTONE ICE CRUISER 5000. Размер 235/55 R1...
id,180f58449e6e2c5cd1e3b985,7d16f82ba22a46a96f4bbd75,19d915b94de2036833f1ff75,1fa174f3d5f95540be332f75
name,Игрушка на батарейках,Комбинезон,"Костюм д/танцев, гимнастики, хореографии на 3-5л.",Зимние шины
payment_available,1,0,0,0
price,1000,300,350,4000
subcategory,314,2206,301,101
fields_g1,Детские товары,Детский гардероб,Детские товары,Транспорт и автотовары


In [21]:
list(c_df['category'].value_counts().index.values)

[9, 22, 3, 12, 1, 4, 14, 10, 8, 15, 5, 11, 2, 13, 6, 17, 100, 16, 7, 20, 21]

In [22]:
c_df[['category', 'fields_g1']].loc[(c_df['category'] == 12)].values

array([[12, 'Телефоны и планшеты'],
       [12, 'Телефоны и планшеты'],
       [12, 'Телефоны и планшеты'],
       ...,
       [12, 'Телефоны и планшеты'],
       [12, 'Телефоны и планшеты'],
       [12, 'Телефоны и планшеты']], dtype=object)

In [23]:
check_cat = lambda cat_num : c_df[['category', 'fields_g1']].loc[(c_df['category'] == cat_num)].values[0] in c_df[['category', 'fields_g1']].loc[(c_df['category'] == cat_num)].values

In [24]:
# CAN WE DROP CATEGORY OR SUBCATEOGRY AND ALL FIELDS AT ALL DUE TO CORRELATION BETWEEN THEM!!! _ YES
result = True
for i in list(c_df['category'].value_counts().index.values):
    result &= check_cat(i)
result    

True

In [25]:
c_df.drop(['fields_g1', 'fields_g2', 'category'], axis=1, inplace=True)

In [26]:
%time
name_descr_preproc(c_df)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


,date_created,delivery_available,description,id,name,payment_available,price,subcategory
0,0.123906,0,не указано,3edeb34cf93f490ff760af85,сумка dg,1,1999.0,914
1,0.113160,0,8 12 лет,c98febd50dad3cc0ffc86085,комплект,0,350.0,2202
2,0.110570,0,на девочку 15 гсостояние хорошее,ade01e13912a46a99134cc75,пальтишко демисезонное,0,300.0,2204
3,0.112810,0,размер 135mm euro 225прочная мягкая не скользя...,ab3e6941c11304c1519aef75,attipas,1,800.0,2209
5,0.131053,0,не указано,252452a91c944a22c276d995,жилет теплый,1,500.0,2204
6,0.127203,0,новые текстильные босоножки 37 размер в размер...,364f7659a4e9ba6c53e35495,босоножки сабо,0,300.0,902
7,0.121136,0,seagate модель st320lt020 hdd для ноутбуков ем...,c3d87065e4dbc080f400bb85,жесткий диск для ноутбука seagate 320gb,1,1500.0,1508
8,0.122741,0,состояние отличноебез дефектов,22e6a518d53f490f6ce7fd85,ковер 324 на 240,1,2000.0,403
10,0.116187,0,продам кенгуру 350 ванночка с горкой 250,c6bd550b7f1304c1f7fda485,кенгуру,0,350.0,301
11,0.114418,0,не указано,117b885a3cf9554042ae2285,машинка автомат,1,3500.0,208


In [27]:
c_df = c_df.set_index('id', drop=True)

In [28]:
c_df.sample(n=4).T

id,021b7be0f0a56f9dbb215d85,fedecc566f40979cdf211685,324c71f6655f2d41297c6585,286cd272d1bc68c6ce777e85
date_created,0.122283,0.117169,0.116714,0.123091
delivery_available,0,0,0,0
description,в отличном состоянии,продам стол в хорошем состоянии самовывоз,продам летнюю резину в хорошем состоянии 175 7...,замечательная стиральная машинкастирает без на...
name,кардан нива,стол компьютерный,продам летнюю резину,стиральная машина hotpoint ariston
payment_available,0,1,0,1
price,2000,2000,4000,12000
subcategory,109,410,101,208


In [29]:
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.externals.joblib import Memory

cache_folder = mkdtemp()
memory_cache = Memory(cachedir=cache_folder, verbose=10)

In [30]:
from sklearn.base import BaseEstimator, TransformerMixin
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, col_name):
        self.col_name = col_name
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.col_name]
    
class ColumnsDropper(BaseEstimator, TransformerMixin):
    def __init__(self, col_names):
        assert isinstance(col_names,list), 'col_names must be a list of exists columns in dataframe'
        assert all([True if type(c) is str else False for c in col_names]), 'col_name must be a string'
        self.col_names = col_names
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.drop(self.col_names, axis=1) #.values ???
        return X
    
def columns_indicies(_df, columns):
    return [_df.columns.get_loc(c) for c in columns]

OHE_COLUMNS_LIST = ['subcategory']

In [52]:
# baking our Pipeline
model = Pipeline([  
    ('union', FeatureUnion([
                    ('name_cv', Pipeline([
                            ('selector', ColumnSelector('name')),
                            ('cv', CountVectorizer(tokenizer=morphy_tokenizer, ngram_range=(1,1)))
                        ]))
                    ,
                    ('descr_cv', Pipeline([
                            ('selector', ColumnSelector('description')),
                            ('cv', CountVectorizer(tokenizer=morphy_tokenizer, ngram_range=(1,1)))
                        ]))
                    ,
                    ('nontext', Pipeline([
                            ('column_dropper', ColumnsDropper(['name', 'description'])),
                            ('scaler', StandardScaler())
                        ]))
                ])
    )
    ,   
    ('rfr', RandomForestRegressor(random_state=42, n_jobs=2))
])

In [53]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import validation_curve

In [54]:
y = c_df['price']
X = c_df.drop(['price'], axis=1)

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
X_train.shape, X_test.shape

((70000, 6), (30000, 6))

In [56]:
param_grid = {
    'rfr__n_estimators' : [50,100],
    'union__nontext__scaler__with_mean' : [True, False]
}
cv = StratifiedKFold(n_splits=4, random_state=42)

In [57]:
grid_search = GridSearchCV(model, param_grid, cv=5, scoring=RMLSE_SCORER)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('name_cv', Pipeline(memory=None,
     steps=[('selector', ColumnSelector(col_name='name')), ('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='con...estimators=10, n_jobs=2,
           oob_score=False, random_state=42, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'rfr__n_estimators': [50, 100], 'union__nontext__scaler__with_mean': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=<function RMLSE_SCORER at 0x7f8decb27400>, verbose=0)

In [101]:
grid_search.best_score_

1.2741342030435598

In [68]:
pickle.dump(grid_search.best_params_, open('GSCV_best_params_.pckl', 'wb'))
grid_search.best_params_

{'rfr__n_estimators': 100, 'union__nontext__scaler__with_mean': True}

In [100]:
grid_search.cv_results_

{'mean_fit_time': array([1865.01206408, 1870.5298738 , 3600.65042038, 3679.90140567]),
 'std_fit_time': array([  4.78928755,   6.65807869,  11.23419386, 130.20112369]),
 'mean_score_time': array([40.10900521, 40.04792919, 40.56554456, 41.1712153 ]),
 'std_score_time': array([0.92434029, 0.46973311, 0.48744926, 1.64087061]),
 'param_rfr__n_estimators': masked_array(data=[50, 50, 100, 100],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_union__nontext__scaler__with_mean': masked_array(data=[True, False, True, False],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'rfr__n_estimators': 50,
   'union__nontext__scaler__with_mean': True},
  {'rfr__n_estimators': 50, 'union__nontext__scaler__with_mean': False},
  {'rfr__n_estimators': 100, 'union__nontext__scaler__with_mean': True},
  {'rfr__n_estimators': 100, 'union__nontext__scaler__with_mean': False}],
 'split0_t

In [98]:
grid_search.grid_scores_

/home/alekum/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 1.26891, std: 0.01652, params: {'rfr__n_estimators': 50, 'union__nontext__scaler__with_mean': True},
 mean: 1.26868, std: 0.01683, params: {'rfr__n_estimators': 50, 'union__nontext__scaler__with_mean': False},
 mean: 1.27413, std: 0.01586, params: {'rfr__n_estimators': 100, 'union__nontext__scaler__with_mean': True},
 mean: 1.27377, std: 0.01600, params: {'rfr__n_estimators': 100, 'union__nontext__scaler__with_mean': False}]

In [67]:
grid_search.best_estimator_

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('name_cv', Pipeline(memory=None,
     steps=[('selector', ColumnSelector(col_name='name')), ('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='con...stimators=100, n_jobs=2,
           oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [71]:
model.set_params(**grid_search.best_params_)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('name_cv', Pipeline(memory=None,
     steps=[('selector', ColumnSelector(col_name='name')), ('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='con...stimators=100, n_jobs=2,
           oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [76]:
# NotFittedError: CountVectorizer - Vocabulary wasn't fitted.
# y_hat = model.predict(X_test) --> because I am stupid it will not work AT ALL! 

In [83]:
cv_name = pickle.load(open('cv_name.pckl', 'rb'))

In [84]:
cv_desc = pickle.load(open('cv_desc.pckl', 'rb'))

In [92]:
prod_model = Pipeline([
      ('union', FeatureUnion([
                    ('name_cv', Pipeline([
                            ('selector', ColumnSelector('name')),
                            ('cv_name', cv_name)
                        ]))
                    ,
                    ('descr_cv', Pipeline([
                            ('selector', ColumnSelector('description')),
                            ('cv_descr', cv_desc)
                        ]))
                    ,
                    ('nontext', Pipeline([
                            ('column_dropper', ColumnsDropper(['name', 'description'])),
                            ('scaler', StandardScaler(with_mean=True))
                        ]))
                ])
    )
    ,   
    ('rfr', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=2))  
    ])

In [93]:
prod_model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('name_cv', Pipeline(memory=None,
     steps=[('selector', ColumnSelector(col_name='name')), ('cv_name', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input...stimators=100, n_jobs=2,
           oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [94]:
y_hat = prod_model.predict(X_test)

In [97]:
RMLSE_SCORE(y_test, y_hat)

1.2237183499919797

In [96]:
pickle.dump(prod_model, open('prod_model_4_sample.pckl', 'wb'))